In [107]:
import pandas as pd
import json
import re
import numpy as np
import ast
from pathlib import Path  

In [108]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/"

In [109]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object']
columns = ['row_path_1','row_path_2','row_path_3','row_path_4']


def process_row_path(row, columns, data_types):
    row['data_type'] = ''
    for column in columns:
        if row[column] in data_types:
            row['data_type'] = row[column]
            row[column] = np.nan
        else:
            row[column]
    return row


In [110]:
def file_paths(df):
    # Create different colums for each part of the document path
    df['path_1'] = df['variable'].str.split('/', n=1).str[0]
    df['path_2'] = df['variable'].str.split('/', n=3).str[1]
    df['path_3'] = df['variable'].str.split('/', n=3).str[2]
    df['path_4'] = df['variable'].str.split('/', n=3).str[3]

    # Create a column with the JSON name
    df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]


    # As the JSON name is stored in the json_name column, fill other parts of the path with Na if the name of the JSON is present
    mark = ".json"

    df['path_2'] = df['path_2'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_3'] = df['path_3'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_4'] = df['path_4'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)

    # Unlist the value column (where the JSON info is stored)
    for index, row in df.iterrows():
        if isinstance(row['value'], list):
            df.at[index, 'value'] = row['value'][0]
    
    return df

In [111]:
def string_to_dict(s):
    result = {}
    items = s.split(',')
    for item in items:
        if ':' not in item:
            continue  
        key, value = item.split(':', 1)  # use maxsplit=1 to avoid unpacking issues
        try:
            key = eval(key.strip())
            value = eval(value.strip())
        except Exception as e:
            continue
        result[key] = value
    return result

    

In [112]:
def row_column_paths(df):
# Initialize the new columns
    df['col_path'] = '' 
    df['row_path_1'] = ''  
    df['row_path_2'] = ''  
    df['row_path_3'] = ''  

    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'col_path'] = list(row['value'].keys())
        
    df['col_path'] = df['col_path'].astype('str').apply(eval).str[0]

    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('col_path')


    # For the level 1 keys stored in col_path, extract the level 2 keys and store in row_path_1 
    df['row_path_1_values'] = df.apply(lambda row: [row['value'].get(row['col_path'], None)], axis=1)
    


    # Unlist and store in individual rows
    df['row_path_1_values'] = df['row_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['row_path_1_values'] = df['row_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    
    """
    # Convert string representations of dict or list to actual dict/list 
    df['row_path_1_values'] = df['row_path_1_values'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.strip().startswith(('{', '[')) else x
    )
    """

    # Assign row_path_1_values to row_path_1
    df['row_path_1'] = df['row_path_1_values']
    df = df.explode('row_path_1')


    df['row_path_1_values'] = df['row_path_1_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)

    # For the level 2 keys stored in row_path_1, extract the level 3 keys and store in row_path_2 
    #df['row_path_2_values'] = df.apply(lambda row: [row['row_path_1_values'].get(row['row_path_1'], None)] if isinstance(row['row_path_1_values'], dict) else None, axis=1)
    df['row_path_2_values'] = df.apply(lambda row: [row['row_path_1_values'].get(row['row_path_1'], None)],axis=1)
    
    
    # Unlist and store in individual rows
    df['row_path_2_values'] = df['row_path_2_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['row_path_2_values'] = df['row_path_2_values'].apply(lambda x: x[0] if isinstance(x, list) else x)


    """
    # Convert string representations of dict or list to actual dict/list (if applicable)
    df['row_path_2_values'] = df['row_path_2_values'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.strip().startswith(('{', '[')) else x
    )
    """
    

    # Assign row_path_2_values to row_path_2
    df['row_path_2'] = df['row_path_2_values']
    df = df.explode('row_path_2')

    df['row_path_2_values'] = df['row_path_2_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    
    

    #ROW_PATH_3
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['row_path_3_values'] = df.apply(lambda row: [row['row_path_2_values'].get(row['row_path_2'], None)] if isinstance (row['row_path_2_values'], dict) else None, axis=1)

    # Unlist and store in individual rows
    df['row_path_3_values'] = df['row_path_3_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['row_path_3_values'] = df['row_path_3_values'].apply(lambda x: x[0] if isinstance(x, list) else x)


    df['row_path_3_values'] = df['row_path_3_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    # Assign row_path_3_values to row_path_3
    df['row_path_3'] = df['row_path_3_values']
    df = df.explode('row_path_3')



    
    #ROW_PATH_4
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['row_path_4_values'] = df.apply(lambda row: [row['row_path_3_values'].get(row['row_path_3'], None)]if isinstance (row['row_path_3_values'], dict) else None, axis=1)

    # Unlist and store in individual rows
    df['row_path_4_values'] = df['row_path_4_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['row_path_4_values'] = df['row_path_4_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    display(df)
    df['row_path_4_values'] = df['row_path_4_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    # Assign row_path_3_values to row_path_3
    df['row_path_4'] = df['row_path_4_values']
    df = df.explode('row_path_4')
    
   

    return df


In [113]:
def clean_and_store(df, file_name):
 
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'col_path', 'row_path_1','row_path_2','row_path_3', 'row_path_4','data_type']]
    
    df.reset_index(drop=True, inplace=True)
    df = df.fillna('Missing')
    #df = df.astype(str)
    


    df.to_csv(f"{main_path}Instagram/Output/Output_" + file_name + '.csv', index=False)

    return df

In [114]:
def structure_donations(data):

    data = Path(data)  
    #file_name = data.name  # Extracts "data.json"
    file_name = Path(data).stem 

    # Load JSON file
    with open(data, 'r') as f:
        data = json.load(f)

    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)

    # Delete user specific informations
    df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)

    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)

    df = file_paths(df)
    
    df = row_column_paths(df)
    
    df = df.apply(lambda row: process_row_path(row, columns, data_types), axis=1)
    
    df = clean_and_store(df, file_name)
    
    
    return df


In [115]:
input_directory = Path(f'{main_path}Instagram/Input')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input


In [116]:

for file in input_directory.iterdir():  
    if file.is_file():  
        print(file)
        structure_donations(file)
        

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/08-04-2025-json_structure instagram.json


,variable,value,path_1,path_2,path_3,path_4,json_name,col_path,row_path_1,row_path_2,row_path_3,row_path_1_values,row_path_2_values,row_path_3_values,row_path_4_values
0,followers_and_following/follow_requests_you've...,{'relationships_follow_requests_received': [{'...,followers_and_following,NaN,NaN,NaN,follow_requests_you've_received.json,relationships_follow_requests_received,title,string,None,"{'title': 'string', 'media_list_data': 'array'...",{},None,None
0,followers_and_following/follow_requests_you've...,{'relationships_follow_requests_received': [{'...,followers_and_following,NaN,NaN,NaN,follow_requests_you've_received.json,relationships_follow_requests_received,media_list_data,array,None,"{'title': 'string', 'media_list_data': 'array'...",{},None,None
0,followers_and_following/follow_requests_you've...,{'relationships_follow_requests_received': [{'...,followers_and_following,NaN,NaN,NaN,follow_requests_you've_received.json,relationships_follow_requests_received,string_list_data,href,NaN,"{'title': 'string', 'media_list_data': 'array'...","{'href': 'string', 'value': 'string', 'timesta...",{},None
0,followers_and_following/follow_requests_you've...,{'relationships_follow_requests_received': [{'...,followers_and_following,NaN,NaN,NaN,follow_requests_you've_received.json,relationships_follow_requests_received,string_list_data,value,NaN,"{'title': 'string', 'media_list_data': 'array'...","{'href': 'string', 'value': 'string', 'timesta...",{},None
0,followers_and_following/follow_requests_you've...,{'relationships_follow_requests_received': [{'...,followers_and_following,NaN,NaN,NaN,follow_requests_you've_received.json,relationships_follow_requests_received,string_list_data,timestamp,NaN,"{'title': 'string', 'media_list_data': 'array'...","{'href': 'string', 'value': 'string', 'timesta...",{},None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,contacts/synced_contacts.json,"{'contacts_contact_info': [{'title': 'string',...",contacts,NaN,NaN,NaN,synced_contacts.json,contacts_contact_info,string_map_data,Contact Information,value,"{'title': 'string', 'media_map_data': {}, 'str...","{'First Name': {'href': 'string', 'value': 'st...","{'href': 'string', 'value': 'string', 'timesta...",string
34,contacts/synced_contacts.json,"{'contacts_contact_info': [{'title': 'string',...",contacts,NaN,NaN,NaN,synced_contacts.json,contacts_contact_info,string_map_data,Contact Information,timestamp,"{'title': 'string', 'media_map_data': {}, 'str...","{'First Name': {'href': 'string', 'value': 'st...","{'href': 'string', 'value': 'string', 'timesta...",number
34,contacts/synced_contacts.json,"{'contacts_contact_info': [{'title': 'string',...",contacts,NaN,NaN,NaN,synced_contacts.json,contacts_contact_info,string_map_data,Imported Time,href,"{'title': 'string', 'media_map_data': {}, 'str...","{'First Name': {'href': 'string', 'value': 'st...","{'href': 'string', 'value': 'string', 'timesta...",string
34,contacts/synced_contacts.json,"{'contacts_contact_info': [{'title': 'string',...",contacts,NaN,NaN,NaN,synced_contacts.json,contacts_contact_info,string_map_data,Imported Time,value,"{'title': 'string', 'media_map_data': {}, 'str...","{'First Name': {'href': 'string', 'value': 'st...","{'href': 'string', 'value': 'string', 'timesta...",string


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/IG_json_structure_CASAS.json


,variable,value,path_1,path_2,path_3,path_4,json_name,col_path,row_path_1,row_path_2,row_path_3,row_path_1_values,row_path_2_values,row_path_3_values,row_path_4_values
0,content/profile_photos.json,"{'ig_profile_picture': [{'uri': 'string', 'cre...",content,NaN,NaN,NaN,profile_photos.json,ig_profile_picture,uri,string,None,"{'uri': 'string', 'creation_timestamp': 'numbe...",{},None,None
0,content/profile_photos.json,"{'ig_profile_picture': [{'uri': 'string', 'cre...",content,NaN,NaN,NaN,profile_photos.json,ig_profile_picture,creation_timestamp,number,None,"{'uri': 'string', 'creation_timestamp': 'numbe...",{},None,None
0,content/profile_photos.json,"{'ig_profile_picture': [{'uri': 'string', 'cre...",content,NaN,NaN,NaN,profile_photos.json,ig_profile_picture,media_metadata,camera_metadata,has_camera_metadata,"{'uri': 'string', 'creation_timestamp': 'numbe...",{'camera_metadata': {'has_camera_metadata': 'b...,{'has_camera_metadata': 'boolean'},boolean
0,content/profile_photos.json,"{'ig_profile_picture': [{'uri': 'string', 'cre...",content,NaN,NaN,NaN,profile_photos.json,ig_profile_picture,title,string,None,"{'uri': 'string', 'creation_timestamp': 'numbe...",{},None,None
0,content/profile_photos.json,"{'ig_profile_picture': [{'uri': 'string', 'cre...",content,NaN,NaN,NaN,profile_photos.json,ig_profile_picture,is_profile_picture,boolean,None,"{'uri': 'string', 'creation_timestamp': 'numbe...",{},None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,contacts/synced_contacts.json,"{'contacts_contact_info': [{'title': 'string',...",contacts,NaN,NaN,NaN,synced_contacts.json,contacts_contact_info,string_map_data,Contact Information,value,"{'title': 'string', 'media_map_data': {}, 'str...","{'First Name': {'href': 'string', 'value': 'st...","{'href': 'string', 'value': 'string', 'timesta...",string
49,contacts/synced_contacts.json,"{'contacts_contact_info': [{'title': 'string',...",contacts,NaN,NaN,NaN,synced_contacts.json,contacts_contact_info,string_map_data,Contact Information,timestamp,"{'title': 'string', 'media_map_data': {}, 'str...","{'First Name': {'href': 'string', 'value': 'st...","{'href': 'string', 'value': 'string', 'timesta...",number
49,contacts/synced_contacts.json,"{'contacts_contact_info': [{'title': 'string',...",contacts,NaN,NaN,NaN,synced_contacts.json,contacts_contact_info,string_map_data,Imported Time,href,"{'title': 'string', 'media_map_data': {}, 'str...","{'First Name': {'href': 'string', 'value': 'st...","{'href': 'string', 'value': 'string', 'timesta...",string
49,contacts/synced_contacts.json,"{'contacts_contact_info': [{'title': 'string',...",contacts,NaN,NaN,NaN,synced_contacts.json,contacts_contact_info,string_map_data,Imported Time,value,"{'title': 'string', 'media_map_data': {}, 'str...","{'First Name': {'href': 'string', 'value': 'st...","{'href': 'string', 'value': 'string', 'timesta...",string


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/json_structure_ig.json


...

,variable,value,path_1,path_2,path_3,path_4,json_name,col_path,row_path_1,row_path_2,row_path_3,row_path_1_values,row_path_2_values,row_path_3_values,row_path_4_values
0,preferences/settings/notification_preferences....,{'settings_notification_preferences': [{'strin...,preferences,settings,NaN,NaN,notification_preferences.json,settings_notification_preferences,string_map_data,Channel,value,{'string_map_data': {'Channel': {'value': 'str...,"{'Channel': {'value': 'string'}, 'Type': {'val...",{'value': 'string'},string
0,preferences/settings/notification_preferences....,{'settings_notification_preferences': [{'strin...,preferences,settings,NaN,NaN,notification_preferences.json,settings_notification_preferences,string_map_data,Type,value,{'string_map_data': {'Channel': {'value': 'str...,"{'Channel': {'value': 'string'}, 'Type': {'val...",{'value': 'string'},string
0,preferences/settings/notification_preferences....,{'settings_notification_preferences': [{'strin...,preferences,settings,NaN,NaN,notification_preferences.json,settings_notification_preferences,string_map_data,Value,value,{'string_map_data': {'Channel': {'value': 'str...,"{'Channel': {'value': 'string'}, 'Type': {'val...",{'value': 'string'},string
1,preferences/settings/use_cross-app_messaging.json,{'settings_upgraded_to_cross_app_messaging': [...,preferences,settings,NaN,NaN,use_cross-app_messaging.json,settings_upgraded_to_cross_app_messaging,title,string,None,"{'title': 'string', 'media_map_data': {}, 'str...",{},None,None
1,preferences/settings/use_cross-app_messaging.json,{'settings_upgraded_to_cross_app_messaging': [...,preferences,settings,NaN,NaN,use_cross-app_messaging.json,settings_upgraded_to_cross_app_messaging,media_map_data,NaN,None,"{'title': 'string', 'media_map_data': {}, 'str...",{},None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,your_instagram_activity/likes/liked_posts.json,"{'likes_media_likes': [{'title': 'string', 'st...",your_instagram_activity,likes,NaN,NaN,liked_posts.json,likes_media_likes,title,string,None,"{'title': 'string', 'string_list_data': [{'hre...",{},None,None
117,your_instagram_activity/likes/liked_posts.json,"{'likes_media_likes': [{'title': 'string', 'st...",your_instagram_activity,likes,NaN,NaN,liked_posts.json,likes_media_likes,string_list_data,href,NaN,"{'title': 'string', 'string_list_data': [{'hre...","{'href': 'string', 'value': 'string', 'timesta...",{},None
117,your_instagram_activity/likes/liked_posts.json,"{'likes_media_likes': [{'title': 'string', 'st...",your_instagram_activity,likes,NaN,NaN,liked_posts.json,likes_media_likes,string_list_data,value,NaN,"{'title': 'string', 'string_list_data': [{'hre...","{'href': 'string', 'value': 'string', 'timesta...",{},None
117,your_instagram_activity/likes/liked_posts.json,"{'likes_media_likes': [{'title': 'string', 'st...",your_instagram_activity,likes,NaN,NaN,liked_posts.json,likes_media_likes,string_list_data,timestamp,NaN,"{'title': 'string', 'string_list_data': [{'hre...","{'href': 'string', 'value': 'string', 'timesta...",{},None


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/08-04-2025-2-json_structure facebook.json


,variable,value,path_1,path_2,path_3,path_4,json_name,col_path,row_path_1,row_path_2,row_path_3,row_path_1_values,row_path_2_values,row_path_3_values,row_path_4_values
0,activity_messages/events_interactions.json,"{'media': 'array', 'label_values': [{'label': ...",activity_messages,NaN,NaN,NaN,events_interactions.json,media,array,None,None,{},None,None,None
1,activity_messages/people_and_friends.json,"{'timestamp': 'number', 'media': 'array', 'lab...",activity_messages,NaN,NaN,NaN,people_and_friends.json,timestamp,number,None,None,{},None,None,None
2,activity_messages/milestone_notifications.json,"{'timestamp': 'number', 'media': 'array', 'lab...",activity_messages,NaN,NaN,NaN,milestone_notifications.json,timestamp,number,None,None,{},None,None,None
3,in-app_messages/your_in-app_messages_interacti...,"{'timestamp': 'number', 'media': 'array', 'lab...",in-app_messages,NaN,NaN,NaN,your_in-app_messages_interactions.json,timestamp,number,None,None,{},None,None,None
4,location/device_location.json,{'phone_number_location_v2': [{'spn': 'string'...,location,NaN,NaN,NaN,device_location.json,phone_number_location_v2,spn,string,None,"{'spn': 'string', 'country_code': 'string'}",{},None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,messages/inbox/facebookuser_10205079055275834/...,"{'participants': [{'name': 'string'}], 'messag...",messages,inbox,facebookuser_10205079055275834,NaN,message_1.json,participants,name,string,None,{'name': 'string'},{},None,None
370,comments_and_reactions/likes_and_reactions.json,"{'timestamp': 'number', 'media': 'array', 'lab...",comments_and_reactions,NaN,NaN,NaN,likes_and_reactions.json,timestamp,number,None,None,{},None,None,None
371,your_activity_off_meta_technologies.json,"{'title': 'string', 'media': 'array', 'label_v...",your_activity_off_meta_technologies.json,NaN,NaN,NaN,your_activity_off_meta_technologies.json,title,string,None,None,{},None,None,None
372,messages/inbox/henkbierlee_1096317910381927/me...,"{'participants': [{'name': 'string'}], 'messag...",messages,inbox,henkbierlee_1096317910381927,NaN,message_1.json,participants,name,string,None,{'name': 'string'},{},None,None


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/nv_insta.json


,variable,value,path_1,path_2,path_3,path_4,json_name,col_path,row_path_1,row_path_2,row_path_3,row_path_1_values,row_path_2_values,row_path_3_values,row_path_4_values
0,followers_and_following/close_friends.json,{'relationships_close_friends': [{'title': 'st...,followers_and_following,NaN,NaN,NaN,close_friends.json,relationships_close_friends,title,string,None,"{'title': 'string', 'media_list_data': 'array'...",{},None,None
0,followers_and_following/close_friends.json,{'relationships_close_friends': [{'title': 'st...,followers_and_following,NaN,NaN,NaN,close_friends.json,relationships_close_friends,media_list_data,array,None,"{'title': 'string', 'media_list_data': 'array'...",{},None,None
0,followers_and_following/close_friends.json,{'relationships_close_friends': [{'title': 'st...,followers_and_following,NaN,NaN,NaN,close_friends.json,relationships_close_friends,string_list_data,href,NaN,"{'title': 'string', 'media_list_data': 'array'...","{'href': 'string', 'value': 'string', 'timesta...",{},None
0,followers_and_following/close_friends.json,{'relationships_close_friends': [{'title': 'st...,followers_and_following,NaN,NaN,NaN,close_friends.json,relationships_close_friends,string_list_data,value,NaN,"{'title': 'string', 'media_list_data': 'array'...","{'href': 'string', 'value': 'string', 'timesta...",{},None
0,followers_and_following/close_friends.json,{'relationships_close_friends': [{'title': 'st...,followers_and_following,NaN,NaN,NaN,close_friends.json,relationships_close_friends,string_list_data,timestamp,NaN,"{'title': 'string', 'media_list_data': 'array'...","{'href': 'string', 'value': 'string', 'timesta...",{},None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,instagram_ads_and_businesses/advertisers_using...,{'ig_custom_audiences_all_types': [{'advertise...,instagram_ads_and_businesses,NaN,NaN,NaN,advertisers_using_your_activity_or_information...,ig_custom_audiences_all_types,advertiser_name,string,None,"{'advertiser_name': 'string', 'has_data_file_c...",{},None,None
97,instagram_ads_and_businesses/advertisers_using...,{'ig_custom_audiences_all_types': [{'advertise...,instagram_ads_and_businesses,NaN,NaN,NaN,advertisers_using_your_activity_or_information...,ig_custom_audiences_all_types,has_data_file_custom_audience,boolean,None,"{'advertiser_name': 'string', 'has_data_file_c...",{},None,None
97,instagram_ads_and_businesses/advertisers_using...,{'ig_custom_audiences_all_types': [{'advertise...,instagram_ads_and_businesses,NaN,NaN,NaN,advertisers_using_your_activity_or_information...,ig_custom_audiences_all_types,has_remarketing_custom_audience,boolean,None,"{'advertiser_name': 'string', 'has_data_file_c...",{},None,None
97,instagram_ads_and_businesses/advertisers_using...,{'ig_custom_audiences_all_types': [{'advertise...,instagram_ads_and_businesses,NaN,NaN,NaN,advertisers_using_your_activity_or_information...,ig_custom_audiences_all_types,has_in_person_store_visit,boolean,None,"{'advertiser_name': 'string', 'has_data_file_c...",{},None,None


In [117]:
# Path to the folder containing CSV files
output_path = f"{main_path}Instagram/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]


common_columns = ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'col_path', 'row_path_1','row_path_2','row_path_3', 'row_path_4','data_type']


merged_df = dfs[0]  # Start with the first DataFrame
for df in dfs[1:]:  # Merge with the rest  #
    merged_df = merged_df.merge(df, on=common_columns, how="outer")




# Filter where col1 contains 'messages', then drop duplicates based on col2
df_filtered = merged_df[merged_df["path_1"] == "messages"].drop_duplicates(subset="path_2")


# Append rows where col1 does not contain 'messages'
df_final = pd.concat([df_filtered, merged_df[merged_df["path_1"] != "messages"]], ignore_index=True)
df_final = df_final.replace('Missing', np.nan)
display(df_final)

print(df_final.isna().sum())

# Save the final merged DataFrame
df_final.to_csv(f"{main_path}Instagram/Final/Merged_structures_IG.csv", index=False)

/tmp/ipykernel_9694/1927761436.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_final.replace('Missing', np.nan)


,variable,value,path_1,path_2,path_3,path_4,json_name,col_path,row_path_1,row_path_2,row_path_3,row_path_4,data_type
0,messages/autofill_information.json,{'autofill_information_v2': {'CITY': ['string'...,messages,NaN,NaN,NaN,autofill_information.json,autofill_information_v2,CITY,NaN,NaN,NaN,string
1,messages/e2ee_cutover/robinvansoest_1016054888...,"{'participants': [{'name': 'string'}], 'messag...",messages,e2ee_cutover,robinvansoest_10160548885986200,NaN,message_1.json,participants,name,NaN,NaN,NaN,string
2,messages/filtered_threads/chatgirl6606_6893657...,"{'participants': [{'name': 'string'}], 'messag...",messages,filtered_threads,chatgirl6606_6893657310699256,NaN,message_1.json,participants,name,NaN,NaN,NaN,string
3,messages/inbox/10153411048572513/message_1.json,"{'participants': [{'name': 'string'}], 'messag...",messages,inbox,10153411048572513,NaN,message_1.json,participants,name,NaN,NaN,NaN,string
4,messages/message_requests/brianajatroband26oth...,"{'participants': [{'name': 'string'}], 'messag...",messages,message_requests,brianajatroband26others_9670767192947684,NaN,message_1.json,participants,name,NaN,NaN,NaN,string
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,your_topics/your_topics.json,"{'topics_your_topics': [{'title': 'string', 'm...",your_topics,NaN,NaN,NaN,your_topics.json,topics_your_topics,media_map_data,NaN,NaN,NaN,NaN
1408,your_topics/your_topics.json,"{'topics_your_topics': [{'title': 'string', 'm...",your_topics,NaN,NaN,NaN,your_topics.json,topics_your_topics,string_map_data,Name,href,NaN,NaN
1409,your_topics/your_topics.json,"{'topics_your_topics': [{'title': 'string', 'm...",your_topics,NaN,NaN,NaN,your_topics.json,topics_your_topics,string_map_data,Name,timestamp,NaN,NaN
1410,your_topics/your_topics.json,"{'topics_your_topics': [{'title': 'string', 'm...",your_topics,NaN,NaN,NaN,your_topics.json,topics_your_topics,string_map_data,Name,value,NaN,NaN


variable         0
value            0
path_1           0
path_2         966
path_3        1348
path_4        1412
json_name        0
col_path         0
row_path_1     130
row_path_2     594
row_path_3     763
row_path_4    1370
data_type      870
dtype: int64
